# Notebook 5: Building Your First MCP Server

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 5 of 13  
**Level:** Intermediate  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Build a complete MCP server using FastMCP
2. Create tools with automatic schema generation
3. Define resources and resource templates
4. Implement prompt templates
5. Add proper type hints and documentation
6. Test your server with MCP Inspector

## 1. Introduction to FastMCP

**FastMCP** is the recommended way to build MCP servers in Python:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    WHY FASTMCP?                                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   🚀 DEVELOPER EXPERIENCE          📝 AUTO-GENERATION                   │
│   ═════════════════════            ════════════════                     │
│   • Decorator-based API            • JSON Schema from type hints        │
│   • Minimal boilerplate            • Descriptions from docstrings       │
│   • Familiar Flask-like style      • Validation automatic               │
│                                                                          │
│   🔧 BATTERIES INCLUDED            🔒 SAFE BY DEFAULT                   │
│   ═══════════════════              ════════════════                     │
│   • STDIO & HTTP transports        • Error handling built-in            │
│   • Async support                  • Type checking                      │
│   • Context management             • Proper logging (stderr)            │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Verify FastMCP is installed
try:
    import fastmcp
    print(f"✅ FastMCP version: {fastmcp.__version__}")
except ImportError:
    print("❌ FastMCP not installed. Run: pip install fastmcp")

# Other imports we'll use
import json
from typing import List, Dict, Optional, Any
from datetime import datetime
from pathlib import Path

## 2. Your First Server: Hello World

Let's start with the simplest possible MCP server:

In [ ]:
# Minimal FastMCP server
HELLO_WORLD_SERVER = '''
#!/usr/bin/env python3
"""Hello World MCP Server

The simplest possible MCP server.

Save as: hello_server.py
Run as:  python hello_server.py
Test:    mcp-inspector python hello_server.py
"""

from fastmcp import FastMCP

# Create the server
mcp = FastMCP("HelloWorld")


@mcp.tool()
def hello(name: str) -> str:
    """Say hello to someone.
    
    Args:
        name: The name of the person to greet
    
    Returns:
        A friendly greeting message
    """
    return f"Hello, {name}! Welcome to MCP! 🎉"


if __name__ == "__main__":
    mcp.run()  # Defaults to STDIO transport
'''

print("Hello World MCP Server:")
print("=" * 60)
print(HELLO_WORLD_SERVER)

### What Happens Behind the Scenes

1. **`FastMCP("HelloWorld")`** - Creates server with name "HelloWorld"
2. **`@mcp.tool()`** - Registers function as an MCP tool
3. **Type hints** (`name: str`) - Converted to JSON Schema
4. **Docstring** - Used for tool description
5. **`mcp.run()`** - Starts the JSON-RPC server

In [ ]:
# The generated JSON Schema for the hello tool
generated_schema = {
    "name": "hello",
    "description": "Say hello to someone.",
    "inputSchema": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the person to greet"
            }
        },
        "required": ["name"]
    }
}

print("Auto-generated Tool Schema:")
print(json.dumps(generated_schema, indent=2))

## 3. Creating Tools with FastMCP

### 3.1 Basic Tools with Type Hints

In [ ]:
# Tools with various type hints
TOOLS_EXAMPLES = '''
from fastmcp import FastMCP
from typing import List, Dict, Optional
from datetime import datetime

mcp = FastMCP("ToolExamples")


# Basic types
@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b


@mcp.tool()
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


# Optional parameters with defaults
@mcp.tool()
def greet(name: str, greeting: str = "Hello", excited: bool = False) -> str:
    """Greet someone with a customizable message.
    
    Args:
        name: Person\'s name
        greeting: The greeting word (default: Hello)
        excited: Add exclamation marks if True
    """
    message = f"{greeting}, {name}"
    return f"{message}!!!" if excited else f"{message}."


# List parameters
@mcp.tool()
def sum_numbers(numbers: List[float]) -> float:
    """Calculate the sum of a list of numbers.
    
    Args:
        numbers: List of numbers to sum
    """
    return sum(numbers)


# Dict return type
@mcp.tool()
def get_stats(numbers: List[float]) -> Dict[str, float]:
    """Calculate statistics for a list of numbers.
    
    Returns dict with min, max, mean, and sum.
    """
    if not numbers:
        return {"error": "Empty list"}
    return {
        "min": min(numbers),
        "max": max(numbers),
        "mean": sum(numbers) / len(numbers),
        "sum": sum(numbers),
        "count": len(numbers)
    }


# Optional parameters (can be None)
@mcp.tool()
def search(
    query: str,
    limit: Optional[int] = None,
    category: Optional[str] = None
) -> List[Dict]:
    """Search for items.
    
    Args:
        query: Search query string
        limit: Maximum results (optional)
        category: Filter by category (optional)
    """
    # Simulated search
    results = [
        {"id": 1, "title": f"Result for {query}", "category": "general"}
    ]
    if limit:
        results = results[:limit]
    return results
'''

print("Tool Examples with Type Hints:")
print("=" * 60)
print(TOOLS_EXAMPLES)

### 3.2 Async Tools

In [ ]:
# Async tools for I/O operations
ASYNC_TOOLS = '''
from fastmcp import FastMCP
import httpx
import asyncio

mcp = FastMCP("AsyncTools")


@mcp.tool()
async def fetch_url(url: str) -> str:
    """Fetch content from a URL.
    
    Args:
        url: The URL to fetch
    
    Returns:
        The response text (first 1000 chars)
    """
    async with httpx.AsyncClient() as client:
        response = await client.get(url, follow_redirects=True)
        response.raise_for_status()
        return response.text[:1000]


@mcp.tool()
async def fetch_multiple(urls: List[str]) -> Dict[str, str]:
    """Fetch content from multiple URLs concurrently.
    
    Args:
        urls: List of URLs to fetch
    
    Returns:
        Dict mapping URL to response text
    """
    async with httpx.AsyncClient() as client:
        async def fetch_one(url: str) -> tuple:
            try:
                response = await client.get(url, follow_redirects=True)
                return (url, response.text[:500])
            except Exception as e:
                return (url, f"Error: {e}")
        
        results = await asyncio.gather(*[fetch_one(url) for url in urls])
        return dict(results)


@mcp.tool()
async def delay_response(message: str, seconds: float = 1.0) -> str:
    """Return a message after a delay (for testing).
    
    Args:
        message: Message to return
        seconds: Delay in seconds
    """
    await asyncio.sleep(seconds)
    return f"After {seconds}s: {message}"
'''

print("Async Tools:")
print("=" * 60)
print(ASYNC_TOOLS)

### 3.3 Tools with Context

FastMCP provides a context object for accessing server state:

In [ ]:
# Tools using context
CONTEXT_TOOLS = '''
from fastmcp import FastMCP, Context
from typing import Annotated

mcp = FastMCP("ContextTools")

# Server-level state
request_count = 0


@mcp.tool()
def get_request_count(ctx: Context) -> int:
    """Get the total number of requests processed.
    
    The Context parameter is automatically injected.
    """
    global request_count
    request_count += 1
    return request_count


@mcp.tool()
async def log_and_echo(ctx: Context, message: str) -> str:
    """Log a message and echo it back.
    
    Args:
        message: Message to log and return
    """
    # Use context for logging
    await ctx.info(f"Processing message: {message}")
    return f"Logged: {message}"


@mcp.tool()
async def report_progress(
    ctx: Context, 
    steps: int = 5
) -> str:
    """Demonstrate progress reporting.
    
    Args:
        steps: Number of steps to simulate
    """
    import asyncio
    
    for i in range(steps):
        await ctx.report_progress(
            progress=i + 1,
            total=steps,
            message=f"Step {i + 1} of {steps}"
        )
        await asyncio.sleep(0.5)
    
    return f"Completed {steps} steps!"
'''

print("Tools with Context:")
print("=" * 60)
print(CONTEXT_TOOLS)

## 4. Creating Resources

Resources provide read-only data to the model:

In [ ]:
# Resource examples
RESOURCES_EXAMPLES = '''
from fastmcp import FastMCP
from datetime import datetime
import json

mcp = FastMCP("ResourceExamples")


# Static resource - fixed URI
@mcp.resource("config://app/settings")
def get_app_settings() -> str:
    """Application configuration settings."""
    settings = {
        "app_name": "MyMCPApp",
        "version": "1.0.0",
        "debug": False,
        "features": ["search", "export", "notifications"]
    }
    return json.dumps(settings, indent=2)


# Dynamic resource - computed on access
@mcp.resource("status://server/health")
def get_server_health() -> str:
    """Current server health status."""
    return json.dumps({
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "uptime_seconds": 3600,
        "memory_usage_mb": 256
    }, indent=2)


# Resource with MIME type
@mcp.resource("data://sample.csv", mime_type="text/csv")
def get_sample_csv() -> str:
    """Sample CSV data."""
    return """id,name,value
1,alpha,100
2,beta,200
3,gamma,300"""


# Resource template - parameterized URI
@mcp.resource("users://{user_id}/profile")
def get_user_profile(user_id: str) -> str:
    """User profile by ID.
    
    Args:
        user_id: The user ID to fetch
    """
    # Simulated user data
    users = {
        "123": {"name": "Alice", "role": "admin"},
        "456": {"name": "Bob", "role": "user"}
    }
    user = users.get(user_id, {"error": "User not found"})
    return json.dumps(user, indent=2)


# Resource template with multiple parameters
@mcp.resource("reports://{year}/{month}")
def get_monthly_report(year: str, month: str) -> str:
    """Monthly report by year and month.
    
    Args:
        year: Report year (e.g., 2025)
        month: Report month (e.g., 01)
    """
    return json.dumps({
        "year": year,
        "month": month,
        "total_sales": 150000,
        "new_customers": 42,
        "generated_at": datetime.now().isoformat()
    }, indent=2)
'''

print("Resource Examples:")
print("=" * 60)
print(RESOURCES_EXAMPLES)

## 5. Creating Prompts

Prompts provide reusable templates for common workflows:

In [ ]:
# Prompt examples
PROMPTS_EXAMPLES = '''
from fastmcp import FastMCP
from fastmcp.prompts import Prompt, Message

mcp = FastMCP("PromptExamples")


# Simple prompt with arguments
@mcp.prompt()
def code_review(language: str, focus: str = "general") -> list:
    """Code review prompt template.
    
    Args:
        language: Programming language of the code
        focus: Review focus area (security, performance, style, general)
    """
    return [
        {
            "role": "user",
            "content": f"""Please review the following {language} code with a focus on {focus}.

For each issue found, provide:
1. Line number or location
2. Description of the issue
3. Severity (high/medium/low)
4. Suggested fix

The code to review will be provided next."""
        }
    ]


# Prompt for data analysis
@mcp.prompt()
def analyze_data(data_type: str, goal: str) -> list:
    """Data analysis prompt template.
    
    Args:
        data_type: Type of data (sales, users, logs, etc.)
        goal: Analysis goal (trends, anomalies, summary, etc.)
    """
    return [
        {
            "role": "user",
            "content": f"""Analyze the following {data_type} data to identify {goal}.

Please provide:
1. Executive summary (2-3 sentences)
2. Key findings with supporting data
3. Visualizations recommendations
4. Actionable recommendations

Use available tools to:
- Fetch the relevant data resources
- Perform calculations as needed
- Generate charts if visualization tools are available"""
        }
    ]


# Multi-step workflow prompt
@mcp.prompt()
def research_topic(topic: str, depth: str = "overview") -> list:
    """Research a topic using available tools.
    
    Args:
        topic: The topic to research
        depth: Research depth (overview, detailed, comprehensive)
    """
    depth_instructions = {
        "overview": "Provide a brief 2-3 paragraph overview.",
        "detailed": "Provide a detailed analysis with sections.",
        "comprehensive": "Provide an exhaustive research report."
    }
    
    return [
        {
            "role": "user",
            "content": f"""Research the topic: {topic}

Depth: {depth_instructions.get(depth, depth_instructions['overview'])}

Steps:
1. Use search tools to find relevant information
2. Fetch and read promising sources
3. Synthesize findings
4. Cite sources properly
5. Identify gaps or areas for further research"""
        }
    ]


# Prompt that includes resource context
@mcp.prompt()
def summarize_document(document_uri: str, format: str = "bullets") -> list:
    """Summarize a document from resources.
    
    Args:
        document_uri: URI of the document resource
        format: Output format (bullets, paragraph, executive)
    """
    return [
        {
            "role": "user",
            "content": f"""Summarize the document at: {document_uri}

Format: {format}

First, fetch the document using resources/read.
Then provide a summary in the requested format."""
        }
    ]
'''

print("Prompt Examples:")
print("=" * 60)
print(PROMPTS_EXAMPLES)

## 6. Complete Example: Note-Taking Server

Let's build a complete, practical MCP server:

In [ ]:
# Complete Note-Taking MCP Server
NOTES_SERVER = '''
#!/usr/bin/env python3
"""Note-Taking MCP Server

A complete MCP server for managing notes with:
- Tools: create, update, delete, search notes
- Resources: list notes, get note by ID
- Prompts: summarize notes, organize notes

Save as: notes_server.py
Run:     python notes_server.py
"""

from fastmcp import FastMCP
from datetime import datetime
from typing import List, Optional, Dict
import json
import uuid

# Create server
mcp = FastMCP(
    "NoteServer",
    description="A server for managing personal notes"
)

# In-memory storage (use database in production)
notes_db: Dict[str, dict] = {}


# ============== TOOLS ==============

@mcp.tool()
def create_note(title: str, content: str, tags: List[str] = None) -> dict:
    """Create a new note.
    
    Args:
        title: Note title
        content: Note content (markdown supported)
        tags: Optional list of tags for organization
    
    Returns:
        The created note with ID and timestamps
    """
    note_id = str(uuid.uuid4())[:8]
    now = datetime.now().isoformat()
    
    note = {
        "id": note_id,
        "title": title,
        "content": content,
        "tags": tags or [],
        "created_at": now,
        "updated_at": now
    }
    
    notes_db[note_id] = note
    return note


@mcp.tool()
def update_note(
    note_id: str,
    title: Optional[str] = None,
    content: Optional[str] = None,
    tags: Optional[List[str]] = None
) -> dict:
    """Update an existing note.
    
    Args:
        note_id: ID of the note to update
        title: New title (optional)
        content: New content (optional)
        tags: New tags (optional)
    
    Returns:
        The updated note
    """
    if note_id not in notes_db:
        return {"error": f"Note {note_id} not found"}
    
    note = notes_db[note_id]
    
    if title is not None:
        note["title"] = title
    if content is not None:
        note["content"] = content
    if tags is not None:
        note["tags"] = tags
    
    note["updated_at"] = datetime.now().isoformat()
    return note


@mcp.tool()
def delete_note(note_id: str) -> dict:
    """Delete a note.
    
    Args:
        note_id: ID of the note to delete
    
    Returns:
        Confirmation of deletion
    """
    if note_id not in notes_db:
        return {"error": f"Note {note_id} not found"}
    
    del notes_db[note_id]
    return {"deleted": note_id, "success": True}


@mcp.tool()
def search_notes(
    query: Optional[str] = None,
    tags: Optional[List[str]] = None,
    limit: int = 10
) -> List[dict]:
    """Search notes by content or tags.
    
    Args:
        query: Text to search in title and content
        tags: Filter by tags (notes must have ALL specified tags)
        limit: Maximum results to return
    
    Returns:
        List of matching notes
    """
    results = []
    
    for note in notes_db.values():
        # Text search
        if query:
            query_lower = query.lower()
            if (query_lower not in note["title"].lower() and 
                query_lower not in note["content"].lower()):
                continue
        
        # Tag filter
        if tags:
            if not all(tag in note["tags"] for tag in tags):
                continue
        
        results.append(note)
        
        if len(results) >= limit:
            break
    
    return results


# ============== RESOURCES ==============

@mcp.resource("notes://all")
def list_all_notes() -> str:
    """List all notes (summary view)."""
    summaries = [
        {
            "id": n["id"],
            "title": n["title"],
            "tags": n["tags"],
            "updated_at": n["updated_at"]
        }
        for n in notes_db.values()
    ]
    return json.dumps(summaries, indent=2)


@mcp.resource("notes://{note_id}")
def get_note(note_id: str) -> str:
    """Get a specific note by ID.
    
    Args:
        note_id: The note ID
    """
    note = notes_db.get(note_id)
    if note:
        return json.dumps(note, indent=2)
    return json.dumps({"error": f"Note {note_id} not found"})


@mcp.resource("notes://tags")
def list_all_tags() -> str:
    """List all unique tags with counts."""
    tag_counts = {}
    for note in notes_db.values():
        for tag in note["tags"]:
            tag_counts[tag] = tag_counts.get(tag, 0) + 1
    return json.dumps(tag_counts, indent=2)


@mcp.resource("notes://stats")
def get_stats() -> str:
    """Get notes statistics."""
    total = len(notes_db)
    total_tags = sum(len(n["tags"]) for n in notes_db.values())
    total_chars = sum(len(n["content"]) for n in notes_db.values())
    
    return json.dumps({
        "total_notes": total,
        "total_tags": total_tags,
        "total_characters": total_chars,
        "avg_length": total_chars // total if total > 0 else 0
    }, indent=2)


# ============== PROMPTS ==============

@mcp.prompt()
def summarize_notes(tag: Optional[str] = None) -> list:
    """Summarize all notes or notes with a specific tag.
    
    Args:
        tag: Optional tag to filter notes
    """
    filter_text = f" with tag '{tag}'" if tag else ""
    
    return [
        {
            "role": "user",
            "content": f"""Summarize all notes{filter_text}.

Steps:
1. First, fetch the notes list using resources/read on notes://all
2. If a tag filter was specified, use search_notes tool to filter
3. Read each relevant note using resources/read on notes://{{note_id}}
4. Provide a summary including:
   - Total notes found
   - Key themes or topics
   - Brief summary of each note (1-2 sentences)
   - Common tags"""
        }
    ]


@mcp.prompt()
def organize_notes() -> list:
    """Suggest organization improvements for notes."""
    return [
        {
            "role": "user",
            "content": """Analyze my notes and suggest organization improvements.

Steps:
1. Fetch all notes using resources/read on notes://all
2. Fetch tag statistics using resources/read on notes://tags
3. Read each note\'s content
4. Suggest:
   - Notes that could be merged
   - Missing tags that should be added
   - Notes that might be outdated
   - Category suggestions for better organization
   - Any duplicate content detected"""
        }
    ]


# Run server
if __name__ == "__main__":
    # Add some sample notes for testing
    create_note(
        "Meeting Notes - Q4 Planning",
        "Discussed roadmap for Q4. Key priorities: launch v2.0, improve docs.",
        ["meetings", "planning", "q4"]
    )
    create_note(
        "Python Best Practices",
        "Use type hints. Write docstrings. Follow PEP8.",
        ["python", "coding", "reference"]
    )
    
    mcp.run()
'''

print("Complete Notes Server:")
print("=" * 60)
print(NOTES_SERVER)

## 7. Testing Your Server

### Using MCP Inspector

```powershell
# Install MCP Inspector
npm install -g @modelcontextprotocol/inspector

# Test your server
mcp-inspector python notes_server.py

# Opens web UI at http://localhost:6274
```

### Manual Testing

In [ ]:
# Test messages for manual testing
test_sequence = [
    {
        "description": "Initialize",
        "message": {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "initialize",
            "params": {
                "protocolVersion": "2025-06-18",
                "capabilities": {},
                "clientInfo": {"name": "TestClient", "version": "1.0"}
            }
        }
    },
    {
        "description": "List Tools",
        "message": {
            "jsonrpc": "2.0",
            "id": 2,
            "method": "tools/list",
            "params": {}
        }
    },
    {
        "description": "Create Note",
        "message": {
            "jsonrpc": "2.0",
            "id": 3,
            "method": "tools/call",
            "params": {
                "name": "create_note",
                "arguments": {
                    "title": "Test Note",
                    "content": "This is a test note.",
                    "tags": ["test", "demo"]
                }
            }
        }
    },
    {
        "description": "List Resources",
        "message": {
            "jsonrpc": "2.0",
            "id": 4,
            "method": "resources/list",
            "params": {}
        }
    }
]

print("Test Sequence for Manual Testing:")
print("=" * 60)
for i, test in enumerate(test_sequence, 1):
    print(f"\n{i}. {test['description']}")
    print(json.dumps(test['message']))

## 8. Configuring Claude Desktop

Add your server to Claude Desktop:

In [ ]:
# Claude Desktop configuration
claude_config = {
    "mcpServers": {
        "notes": {
            "command": "python",
            "args": ["C:\\Users\\YourName\\mcp-servers\\notes_server.py"],
            "env": {
                "PYTHONUNBUFFERED": "1"
            }
        }
    }
}

print("Claude Desktop Configuration:")
print("=" * 60)
print(f"Save to: %APPDATA%\\Claude\\claude_desktop_config.json")
print()
print(json.dumps(claude_config, indent=2))

## 9. Key Takeaways

### FastMCP Best Practices

1. **Use type hints** - They generate JSON Schema automatically
2. **Write docstrings** - They become tool/resource descriptions
3. **Use Optional for optional params** - With default values
4. **Async for I/O** - Use async functions for network/file operations
5. **Context for state** - Use Context parameter for logging, progress

### Server Structure

```python
from fastmcp import FastMCP

mcp = FastMCP("ServerName")

@mcp.tool()
def my_tool(...): ...  # Tools for actions

@mcp.resource("uri://...")
def my_resource(): ...  # Resources for data

@mcp.prompt()
def my_prompt(...): ...  # Prompts for workflows

if __name__ == "__main__":
    mcp.run()
```

### Next Steps

In **Notebook 6: Workshop - Filesystem MCP Server**, we'll:
- Build a complete filesystem server from scratch
- Implement secure file access
- Add directory navigation and search
- Configure for Claude Desktop

## 10. Exercises

### Exercise 1: Add a Tool

Add a `get_note_history` tool to the notes server that tracks changes.

In [ ]:
# Exercise 1: Implement note history tracking
HISTORY_TOOL = '''
# Add to notes_server.py

# Storage for history
notes_history: Dict[str, List[dict]] = {}

@mcp.tool()
def get_note_history(note_id: str) -> List[dict]:
    """Get the change history for a note.
    
    Args:
        note_id: ID of the note
    
    Returns:
        List of historical versions with timestamps
    """
    # TODO: Implement
    # 1. Check if note exists
    # 2. Return history or empty list
    pass
'''

print("Exercise 1: Implement the history tool")
print(HISTORY_TOOL)

### Exercise 2: Add a Resource Template

Add a resource template for fetching notes by tag.

In [ ]:
# Exercise 2: Notes by tag resource
TAG_RESOURCE = '''
@mcp.resource("notes://tags/{tag_name}")
def get_notes_by_tag(tag_name: str) -> str:
    """Get all notes with a specific tag.
    
    Args:
        tag_name: The tag to filter by
    """
    # TODO: Implement
    # 1. Filter notes_db by tag
    # 2. Return JSON list of matching notes
    pass
'''

print("Exercise 2: Implement the tag filter resource")
print(TAG_RESOURCE)

### Exercise 3: Create Your Own Server

Design and implement a simple MCP server for one of these:
- Todo list manager
- Bookmark organizer
- Weather data server
- Calculator with history

In [ ]:
# Exercise 3: Your server template
YOUR_SERVER = '''
from fastmcp import FastMCP

mcp = FastMCP("YourServerName")

# TODO: Add your tools, resources, and prompts

if __name__ == "__main__":
    mcp.run()
'''

print("Exercise 3: Create your own server!")
print(YOUR_SERVER)

---

## References

- [FastMCP Documentation](https://github.com/jlowin/fastmcp)
- [MCP Server Specification](https://modelcontextprotocol.io/docs/spec/server)
- [Python Type Hints](https://docs.python.org/3/library/typing.html)
- [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector)

---

**Next Notebook:** [06_Workshop_Filesystem_MCP_Server.ipynb](./06_Workshop_Filesystem_MCP_Server.ipynb)